# Дошашнее задание

**ПАКЕТ SURPRISE**

**используйте данные MovieLens 1M
можно использовать любые модели из пакета
получите RMSE на тестовом сете 0.87 и ниже
Комментарий преподавателя :
В ДЗ на датасет 1М может не хватить RAM. Можно сделать на 100K. Качество RMSE предлагаю считать на основе CrossValidation (5 фолдов), а не отложенном датасете.**

In [50]:
from surprise import KNNWithMeans, KNNBasic, KNNBaseline
from surprise import Dataset
from surprise import accuracy
from surprise import Reader
from surprise.model_selection import train_test_split
from surprise.model_selection import cross_validate
from surprise.model_selection import PredefinedKFold
from surprise.model_selection import KFold
from surprise import SVD

import pandas as pd

In [24]:
links = pd.read_csv('C:/Users/AAZam/Documents/Lern DS of Netology/Курс ML/12_Введение и классификация рекомендательных систем/links.csv')
movies = pd.read_csv('C:/Users/AAZam/Documents/Lern DS of Netology/Курс ML/12_Введение и классификация рекомендательных систем/movies.csv')
ratings = pd.read_csv('C:/Users/AAZam/Documents/Lern DS of Netology/Курс ML/12_Введение и классификация рекомендательных систем/ratings.csv')
tags = pd.read_csv('C:/Users/AAZam/Documents/Lern DS of Netology/Курс ML/12_Введение и классификация рекомендательных систем/tags.csv')

In [25]:
ratings.head()

,userId,movieId,rating,timestamp
0,1,1,4.0,964982703
1,1,3,4.0,964981247
2,1,6,4.0,964982224
3,1,47,5.0,964983815
4,1,50,5.0,964982931


In [26]:
movies.head()

,movieId,title,genres
0,1,Toy Story (1995),Adventure|Animation|Children|Comedy|Fantasy
1,2,Jumanji (1995),Adventure|Children|Fantasy
2,3,Grumpier Old Men (1995),Comedy|Romance
3,4,Waiting to Exhale (1995),Comedy|Drama|Romance
4,5,Father of the Bride Part II (1995),Comedy


In [27]:
movies_with_ratings = movies.join(ratings.set_index('movieId'), on='movieId').reset_index(drop=True)
movies_with_ratings.dropna(inplace=True)

In [28]:
movies_with_ratings.head()

,movieId,title,genres,userId,rating,timestamp
0,1,Toy Story (1995),Adventure|Animation|Children|Comedy|Fantasy,1.0,4.0,9.649827e+08
1,1,Toy Story (1995),Adventure|Animation|Children|Comedy|Fantasy,5.0,4.0,8.474350e+08
2,1,Toy Story (1995),Adventure|Animation|Children|Comedy|Fantasy,7.0,4.5,1.106636e+09
3,1,Toy Story (1995),Adventure|Animation|Children|Comedy|Fantasy,15.0,2.5,1.510578e+09
4,1,Toy Story (1995),Adventure|Animation|Children|Comedy|Fantasy,17.0,4.5,1.305696e+09


In [29]:
movies_with_ratings[movies_with_ratings.userId == 10.0].title.unique().len()

array(['Toy Story (1995)', 'Usual Suspects, The (1995)',
       'Postman, The (Postino, Il) (1994)', 'Apollo 13 (1995)',
       'Die Hard: With a Vengeance (1995)',
       'Star Wars: Episode IV - A New Hope (1977)', 'Forrest Gump (1994)',
       'True Lies (1994)', 'Hot Shots! Part Deux (1993)',
       'Jurassic Park (1993)', 'Ghost (1990)', 'Aladdin (1992)',
       'Terminator 2: Judgment Day (1991)', 'Batman (1989)',
       'Silence of the Lambs, The (1991)', 'Beauty and the Beast (1991)',
       'Mission: Impossible (1996)',
       'Dr. Strangelove or: How I Learned to Stop Worrying and Love the Bomb (1964)',
       'Independence Day (a.k.a. ID4) (1996)',
       'Gone with the Wind (1939)', '2001: A Space Odyssey (1968)',
       'Top Gun (1986)', 'English Patient, The (1996)',
       'Star Wars: Episode V - The Empire Strikes Back (1980)',
       'Apocalypse Now (1979)',
       'Star Wars: Episode VI - Return of the Jedi (1983)',
       'Psycho (1960)', 'Blues Brothers, The (1980)'

In [30]:
dataset = pd.DataFrame({
    'uid': movies_with_ratings.userId,
    'iid': movies_with_ratings.title,
    'rating': movies_with_ratings.rating
})

In [31]:
dataset.head()

,uid,iid,rating
0,1.0,Toy Story (1995),4.0
1,5.0,Toy Story (1995),4.0
2,7.0,Toy Story (1995),4.5
3,15.0,Toy Story (1995),2.5
4,17.0,Toy Story (1995),4.5


In [32]:
ratings.rating.min()

0.5

In [33]:
ratings.rating.max()

5.0

In [34]:
reader = Reader(rating_scale=(0.5, 5.0))
data = Dataset.load_from_df(dataset, reader)

In [35]:
data

In [36]:
algo = KNNWithMeans(k=50, sim_options={'name': 'pearson_baseline', 'user_based': True})

In [37]:
# Запустим 5-fold кросс-валидацию и выведем результаты
cross_validate(algo, data, measures=['RMSE'], cv=5, verbose=True)

Estimating biases using als...
Computing the pearson_baseline similarity matrix...
Done computing similarity matrix.
Estimating biases using als...
Computing the pearson_baseline similarity matrix...
Done computing similarity matrix.
Estimating biases using als...
Computing the pearson_baseline similarity matrix...
Done computing similarity matrix.
Estimating biases using als...
Computing the pearson_baseline similarity matrix...
Done computing similarity matrix.
Estimating biases using als...
Computing the pearson_baseline similarity matrix...
Done computing similarity matrix.
Evaluating RMSE of algorithm KNNWithMeans on 5 split(s).

                  Fold 1  Fold 2  Fold 3  Fold 4  Fold 5  Mean    Std     
RMSE (testset)    0.8916  0.8991  0.9011  0.9009  0.8888  0.8963  0.0051  
Fit time          1.00    1.09    1.10    0.93    0.92    1.01    0.08    
Test time         1.53    1.50    1.55    1.33    1.43    1.47    0.08    


{'test_rmse': array([0.89158376, 0.89912292, 0.90108626, 0.90092406, 0.88884826]),
 'fit_time': (0.9989116191864014,
  1.0946860313415527,
  1.0988407135009766,
  0.9252760410308838,
  0.9180057048797607),
 'test_time': (1.5278708934783936,
  1.5048418045043945,
  1.54518723487854,
  1.3258731365203857,
  1.4257898330688477)}

In [ ]:
algo = KNNWithMeans(k=50, sim_options={'name': 'pearson_baseline', 'user_based': True})

## Попробуем еще один вариант кросс-валидации

In [43]:
# define a cross-validation iterator
kf = KFold(n_splits=5)

algo = SVD()

for trainset, testset in kf.split(data):

    # train and test algorithm.
    algo.fit(trainset)
    predictions = algo.test(testset)

    # Compute and print Root Mean Squared Error
    accuracy.rmse(predictions, verbose=True)

RMSE: 0.8819
RMSE: 0.8767
RMSE: 0.8696
RMSE: 0.8724
RMSE: 0.8727


In [46]:
# define a cross-validation iterator
kf = KFold(n_splits=5)

algo = KNNWithMeans(k=50, sim_options={'name': 'pearson_baseline', 'user_based': False})

for trainset, testset in kf.split(data):

    # train and test algorithm.
    algo.fit(trainset)
    predictions = algo.test(testset)

    # Compute and print Root Mean Squared Error
    accuracy.rmse(predictions, verbose=True)

Estimating biases using als...
Computing the pearson_baseline similarity matrix...
Done computing similarity matrix.
RMSE: 0.8815
Estimating biases using als...
Computing the pearson_baseline similarity matrix...
Done computing similarity matrix.
RMSE: 0.8728
Estimating biases using als...
Computing the pearson_baseline similarity matrix...
Done computing similarity matrix.
RMSE: 0.8854
Estimating biases using als...
Computing the pearson_baseline similarity matrix...
Done computing similarity matrix.
RMSE: 0.8830
Estimating biases using als...
Computing the pearson_baseline similarity matrix...
Done computing similarity matrix.
RMSE: 0.8850


In [47]:
# define a cross-validation iterator
kf = KFold(n_splits=5)

algo = KNNWithMeans(k=50, sim_options={'name': 'cosine', 'user_based': False})

for trainset, testset in kf.split(data):

    # train and test algorithm.
    algo.fit(trainset)
    predictions = algo.test(testset)

    # Compute and print Root Mean Squared Error
    accuracy.rmse(predictions, verbose=True)

Computing the cosine similarity matrix...
Done computing similarity matrix.
RMSE: 0.8997
Computing the cosine similarity matrix...
Done computing similarity matrix.
RMSE: 0.9000
Computing the cosine similarity matrix...
Done computing similarity matrix.
RMSE: 0.9055
Computing the cosine similarity matrix...
Done computing similarity matrix.
RMSE: 0.9038
Computing the cosine similarity matrix...
Done computing similarity matrix.
RMSE: 0.9050


In [77]:
from statistics import mean

# define a cross-validation iterator
kf = KFold(n_splits=5)

algo = KNNBaseline(k=30, sim_options={'name': 'pearson_baseline', 'user_based': False})

accuracy_n = []
n = 0
for trainset, testset in kf.split(data):

    # train and test algorithm.
    algo.fit(trainset)
    predictions = algo.test(testset, verbose=False)
    n += 1
    # Compute and print Root Mean Squared Error
    accuracy_n.append(accuracy.rmse(predictions, verbose=False))
#     print(accuracy_n)
#     break
    print(f'Среднеквадратичная погрешность {n}-го измерения равна {round(accuracy.rmse(predictions, verbose=False),2)}')
print(f'Среднеквадратичная погрешность в среднем равна {round(mean(accuracy_n), 2)} \r')

Estimating biases using als...
Computing the pearson_baseline similarity matrix...
Done computing similarity matrix.
Среднеквадратичная погрешность 1-го измерения равна 0.85
Estimating biases using als...
Computing the pearson_baseline similarity matrix...
Done computing similarity matrix.
Среднеквадратичная погрешность 2-го измерения равна 0.86
Estimating biases using als...
Computing the pearson_baseline similarity matrix...
Done computing similarity matrix.
Среднеквадратичная погрешность 3-го измерения равна 0.85
Estimating biases using als...
Computing the pearson_baseline similarity matrix...
Done computing similarity matrix.
Среднеквадратичная погрешность 4-го измерения равна 0.85
Estimating biases using als...
Computing the pearson_baseline similarity matrix...
Done computing similarity matrix.
Среднеквадратичная погрешность 5-го измерения равна 0.85
Среднеквадратичная погрешность в среднем равна 0.85 


In [69]:
algo_1 = KNNBaseline(k=30, sim_options={'name': 'pearson_baseline', 'user_based': False})
cross_validate(algo_1, data, measures=['RMSE'], cv=5, verbose=True)

Estimating biases using als...
Computing the pearson_baseline similarity matrix...
Done computing similarity matrix.
Estimating biases using als...
Computing the pearson_baseline similarity matrix...
Done computing similarity matrix.
Estimating biases using als...
Computing the pearson_baseline similarity matrix...
Done computing similarity matrix.
Estimating biases using als...
Computing the pearson_baseline similarity matrix...
Done computing similarity matrix.
Estimating biases using als...
Computing the pearson_baseline similarity matrix...
Done computing similarity matrix.
Evaluating RMSE of algorithm KNNBaseline on 5 split(s).

                  Fold 1  Fold 2  Fold 3  Fold 4  Fold 5  Mean    Std     
RMSE (testset)    0.8475  0.8532  0.8518  0.8553  0.8614  0.8538  0.0045  
Fit time          14.97   14.82   13.98   13.89   14.87   14.51   0.47    
Test time         31.60   24.06   8.24    22.77   22.74   21.88   7.57    


{'test_rmse': array([0.84749798, 0.85317435, 0.85183376, 0.85532488, 0.86136418]),
 'fit_time': (14.96857213973999,
  14.823780298233032,
  13.979823112487793,
  13.890343427658081,
  14.870306253433228),
 'test_time': (31.59602999687195,
  24.06184411048889,
  8.236443519592285,
  22.77282476425171,
  22.73795986175537)}

### Добились поставленной задачи - RMSE на тестовом сете ниже 0.87